Installieren und Laden benötigter Bibliotheken sowie des Git-Repositories

In [ ]:
!git clone https://github.com/JanKrauland/Masterthesis_Jan-Krauland.git

Cloning into 'PlantVillage_Tomato'...
remote: Enumerating objects: 47009, done.
remote: Total 47009 (delta 0), reused 0 (delta 0), pack-reused 47009
Receiving objects: 100% (47009/47009), 2.08 GiB | 27.68 MiB/s, done.
Resolving deltas: 100% (222/222), done.
Updating files: 100% (251205/251205), done.


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-om2npeil
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-om2npeil
  Resolved https://github.com/facebookresearch/detectron2.git to commit a0e22dbfa791e6235e4f196d5ce25e754d02be31
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6121755 sha256=67ad5312f0427d738fa4d62984966

In [ ]:
import json
import os

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.config import get_cfg as _get_cfg
from detectron2 import model_zoo
import torch.cuda
import torch
import cv2

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA verfügbar, Anzahl der GPUs:", torch.cuda.device_count())
else:
    print("CUDA nicht verfügbar")

CUDA verfügbar, Anzahl der GPUs: 1


Trainingsvorbereitung

In [ ]:
# Wechsel des Arbeitsverzeichnisses, damit die einzelnen Trainingsskripte (.py) ausgeführt werden können
%cd /content/Masterthesis_Jan-Krauland/Detectron2_train/Code

/content/drive/MyDrive/Detectron2


In [ ]:
# Diese Funktion ist notwendig, da die Annotationen nicht in Google Colab erstellt wurden und die Dateipfade in den Annotationsdateien angepasst werden müssen
def path_converter(old_path, new_path):
  json_file_path = old_path

  with open(json_file_path, "r") as json_file:
      data = json.load(json_file)

  for entry in data:
      current_file_name = entry["file_name"]

      parts = current_file_name.split("\\")
      new_file_name = parts[-1]
      new_file_name = new_path + "/" + new_file_name
      entry["file_name"] = new_file_name

  with open(json_file_path, "w") as json_file:
      json.dump(data, json_file, indent=4)

for k in range(5):
  train_json = "/content/Masterthesis_Jan-Krauland/K-Fold_PlantDoc/Split-" + str(k+1) + "/train_annotations.json"
  test_json = "/content/Masterthesis_Jan-Krauland/K-Fold_PlantDoc/Split-" + str(k+1) + "/test_annotations.json"
  new_path_train = "/content/Masterthesis_Jan-Krauland/K-Fold_PlantDoc/Split-" + str(k+1) + "/train/images"
  new_path_test = "/content/Masterthesis_Jan-Krauland/K-Fold_PlantDoc/Split-" + str(k+1) + "/test/images"

  print(new_path_train)
  path_converter(train_json, new_path_train)
  path_converter(test_json, new_path_test)

original_test_json = "/content/Masterthesis_Jan-Krauland/PlantDoc_Test/test_annotations.json"
new_path_original_test = "/content/Masterthesis_Jan-Krauland/PlantDoc_Test" + "/test/images"
path_converter(original_test_json, new_path_original_test)

/content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-1/train/images
/content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-2/train/images
/content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-3/train/images
/content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-4/train/images
/content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-5/train/images


Training der Modelle mit K-Fold Cross-Validation

In [ ]:
num_splits = 5

base_command = "python train.py --batch-size 4 --device gpu --iterations 6750 --checkpoint-period 150 --model 'COCO-Detection/retinanet_R_50_FPN_3x.yaml'"
plot_loss_command = "python plot_loss.py"

for split in range(1, num_splits + 1):
    data_dir = f"/content/Masterthesis_Jan-Krauland/K-Fold_PlantDoc/Split-{split}"
    output_dir = f"/content/drive/MyDrive/Detectron2/Retinanet_R50_FPN_50Epochs_K-Fold_LRScheduler0.0002/Split-{split}"  # Hier den Pfad zum Output-Ordner anpassen
    os.makedirs(output_dir, exist_ok=True)
    command = f"{base_command} --data-dir {data_dir} --output-dir {output_dir}"

    !{command}

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.196
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.173
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.471
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.529
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.447
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.570
[10/27 11:34:37 d2.evaluation.coco_evaluation]: Evaluation results for bbox: 
|   AP   |  AP50  |  AP75  |  APs  |  APm   |  APl   |
|:------:|:------:|:------:|:-----:|:------:|:------:|
| 16.784 | 29.660 | 17.098 | 1.042 | 11.341 | 19.614 |
[10/27 11:34:37 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category                   | AP     | category           

Evaluation

In [ ]:
# Funktion zum extrahieren der Annotationen für die Evaluation
def get_dicts(json_path):
    with open(json_path, "r") as f:
        annotation_data = json.load(f)

    dataset_dicts = []
    for image_info in annotation_data:
        record = {}
        record["file_name"] = image_info["file_name"]
        record["image_id"] = image_info["image_id"]
        record["height"] = image_info["height"]
        record["width"] = image_info["width"]

        annotations = image_info.get("annotations", [])
        objs = []
        for annotation in annotations:
            obj = {
                "bbox": annotation["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": annotation["category_id"],
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts

In [ ]:
# Für jedes Modell wird eine Evaluierung auf dem Testdatensatz durchgeführt

models_to_eval = [
    "/content/drive/MyDrive/Detectron2/Faster_RCNN_R50_FPN_50Epochs_K-Fold_Default",
    "/content/drive/MyDrive/Detectron2/Faster_RCNN_R50_FPN_50Epochs_K-Fold_LRScheduler0.0004",
    "/content/drive/MyDrive/Detectron2/Faster_Retinanet_R50_FPN_50Epochs_K-Fold_Default",
    "/content/drive/MyDrive/Detectron2/Retinanet_R50_FPN_50Epochs_K-Fold_LRScheduler0.0002"
]
configs_to_eval = [
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
    "COCO-Detection/retinanet_R_50_FPN_3x.yaml",
    "COCO-Detection/retinanet_R_50_FPN_3x.yaml"
]

for model_path, config_path in zip(models_to_eval, configs_to_eval):
    print(f"Aktuell wird folgendes Modell evaluiert: {model_path}")
    print(f"Die zugehörige Konfiguration ist: {config_path}")

    for i in range(5):
      if "test" in DatasetCatalog.list():
          DatasetCatalog.remove("test")

      DatasetCatalog.register('test', lambda: get_dicts(r"/content/Masterthesis_Jan-Krauland/PlantDoc_Test/test_annotations.json"))
      MetadataCatalog.get('test').set(thing_classes=['Tomato Early blight leaf', 'Tomato leaf', 'Tomato leaf late blight', 'Tomato leaf bacterial spot', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato Septoria leaf spot'])

      cfg = get_cfg()
      cfg.merge_from_file(model_zoo.get_config_file(config_path))
      cfg.MODEL.WEIGHTS = model_path + "/Split-" + str(i+1) + "/model_final.pth"
      cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
      cfg.OUTPUT_DIR = model_path + "/Split-" + str(i+1) + "/Final_Test"

      predictor = DefaultPredictor(cfg)

      evaluator = COCOEvaluator("test", cfg, True, output_dir=cfg.OUTPUT_DIR)
      val_loader = build_detection_test_loader(cfg, "test")

      print(inference_on_dataset(predictor.model, val_loader, evaluator))


Aktuell wird folgendes Modell evaluiert: /content/drive/MyDrive/Detectron2/Faster_RCNN_R50_FPN_50Epochs_K-Fold_Default
Die zugehörige Konfiguration ist: COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.263
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.139
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.148
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.198
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.308
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.189
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.196
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.409
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.415
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.150
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.318
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.165
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.170
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.376
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.220
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.157
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.149
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.344
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.349
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.152
 Average Recall     (AR) @[ IoU=0.50:0.

...

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.333
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.264
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.561
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.587
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.424
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.204
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.333
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.214
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.255
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.573
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.602
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.606
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.323
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.190
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.264
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.558
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.619
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.315
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.220
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.257
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.573
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.597
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.303
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.180
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.193
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.270
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.566
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.586
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.451
 Average Recall     (AR) @[ IoU=0.50:0.

In [ ]:
if "test" in DatasetCatalog.list():
  DatasetCatalog.remove("test")

DatasetCatalog.register('test', lambda: get_dicts(r"/content/Masterthesis_Jan-Krauland/PlantDoc_Test/test_annotations.json"))
MetadataCatalog.get('test').set(thing_classes=['Tomato Early blight leaf', 'Tomato leaf', 'Tomato leaf late blight', 'Tomato leaf bacterial spot', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato Septoria leaf spot'])

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Detectron2/Faster_RCNN_R50_FPN_50Epochs_K-Fold_Default/Split-1/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.OUTPUT_DIR = "/content/drive/MyDrive/Detectron2/Faster_RCNN_R50_FPN_50Epochs_K-Fold_Default/Split-1/Final_Test"

predictor = DefaultPredictor(cfg)

evaluator = COCOEvaluator("test", cfg, True, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "test")

inference_on_dataset(predictor.model, val_loader, evaluator)


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.263
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.139
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.148
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.198
 Average Recall     (AR) @[ IoU=0.50:0.

OrderedDict([('bbox',
              {'AP': 14.135719148242018,
               'AP50': 26.25012963429877,
               'AP75': 13.234882730274396,
               'APs': 7.087458745874585,
               'APm': 12.117913013920607,
               'APl': 13.850895988339643,
               'AP-Tomato Early blight leaf': 21.01937789963757,
               'AP-Tomato leaf': 16.36103995145059,
               'AP-Tomato leaf late blight': 0.4298280525345397,
               'AP-Tomato leaf bacterial spot': 1.978582632456794,
               'AP-Tomato leaf mosaic virus': 16.44199751043345,
               'AP-Tomato leaf yellow virus': 27.48843498084643,
               'AP-Tomato mold leaf': 4.609104413534899,
               'AP-Tomato Septoria leaf spot': 24.757387745041893})])